<a href="https://colab.research.google.com/github/Rdalzell12/Bayesian-Analysis-of-the-Higgs-Boson-Discovery/blob/main/Copy_of_8_TeV_Initial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Connecting Colab to Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Importing 8 TeV data

!ls /content/drive/MyDrive/ULAB/complete_set_of_ATLAS_open_data_samples_July_2016.zip

/content/drive/MyDrive/ULAB/complete_set_of_ATLAS_open_data_samples_July_2016.zip


In [ ]:
#Unzipping the file

import os
zip_path = '/content/drive/MyDrive/ULAB/complete_set_of_ATLAS_open_data_samples_July_2016.zip'
extract_path = '/content/local_data/'
os.makedirs(extract_path, exist_ok=True)
!unzip "$zip_path" -d "$extract_path"

Archive:  /content/drive/MyDrive/ULAB/complete_set_of_ATLAS_open_data_samples_July_2016.zip
  inflating: /content/local_data/.gitignore  
   creating: /content/local_data/MC/
  inflating: /content/local_data/MC/mc_167747.WtaunuJetsBVeto.root  
  inflating: /content/local_data/MC/mc_110905.ZPrime1500.root  
  inflating: /content/local_data/MC/mc_110904.ZPrime1250.root  
  inflating: /content/local_data/MC/mc_110906.ZPrime1750.root  
  inflating: /content/local_data/MC/mc_110901.ZPrime500.root  
  inflating: /content/local_data/MC/mc_167742.WenuNoJetsBVeto.root  
  inflating: /content/local_data/MC/mc_110902.ZPrime750.root  
  inflating: /content/local_data/MC/mc_167741.WenuJetsBVeto.root  
  inflating: /content/local_data/MC/mc_173045.DYtautauM08to15.root  
  inflating: /content/local_data/MC/mc_117050.ttbar_lep.root  
  inflating: /content/local_data/MC/mc_160205.VBFH125_ZZ4lep.root  
  inflating: /content/local_data/MC/mc_160155.ggH125_ZZ4lep.root  
  inflating: /content/local_data/MC

In [ ]:
#Checking what branches the zip file had
import os
os.listdir('/content/local_data')

#The main file has three branches, as listed below - Data, MC, and .gitignore. Data and MC are branches containing relevant .root files,
#while .gitignore is a file the computer needs to run - not important to us.

['Data', 'MC', '.gitignore']

In [ ]:
#Checking what is in the 'Data' branch
import os
os.listdir('/content/local_data/Data')

['DataMuons.root', 'DataEgamma.root']

In [ ]:
#Variable descriptions of Muon data

!pip install uproot

import uproot

file_path = '/content/local_data/Data/DataMuons.root'

try:
    with uproot.open(file_path) as file:
        if 'mini;1' in file:
            tree = file['mini;1']
            print("Description of variables (branches) in TTree 'mini;1':\n")
            for branch_name in tree.keys():
                try:
                    branch_interpretation = tree[branch_name].interpretation.typename
                    print(f"  - {branch_name}: {branch_interpretation}")
                except Exception as branch_e:
                    print(f"  - {branch_name}: Could not determine type (Error: {branch_e})")
        else:
            print(f"Error: TTree 'mini;1' not found in {file_path}")
except Exception as e:
    print(f"Error opening or processing the file: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.8/393.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.7/913.7 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.7/656.7 kB 39.3 MB/s eta 0:00:00
Description of variables (branches) in TTree 'mini;1':

  - runNumber: int32_t
  - eventNumber: int32_t
  - channelNumber: int32_t
  - mcWeight: float
  - pvxp_n: int32_t
  - vxp_z: float
  - scaleFactor_PILEUP: float
  - scaleFactor_ELE: float
  - scaleFactor_MUON: float
  - scaleFactor_BTAG: float
  - scaleFactor_TRIGGER: float
  - scaleFactor_JVFSF: float
  - scaleFactor_ZVERTEX: float
  - trigE: bool
  - trigM: bool
  - passGRL: bool
  - hasGoodVertex: bool
  - lep_n: uint32_t
  - lep_truthMatched: bool[]
  - lep_trigMatched: uint16_t[]
  - lep_pt: float[]
  - lep_eta: float[]
  - lep_phi: float[]
  - lep_E: float[]
  - lep_z0: float[]
  - lep_charge: float[]
  - lep_type: uint32_t[]
  - lep_flag: uint32_t[]
  - lep_ptcone30: float[]
  - lep_e

In [ ]:
import uproot
import awkward as ak

file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    'lep_type'
]

#Seeing the shape (number of leptons/event) of the data - this shape should be the same for every file, as each file is a different set of data describing the same set of events,
#so this cell only needs to be run once - takes forever to run

try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    row_lengths = [len(row) for row in extracted_data['lep_type']]

except Exception as e:
    print(f"Error processing data: {e}")

one_lep = row_lengths.count(1)
two_lep = row_lengths.count(2)
three_lep = row_lengths.count(3)
four_lep = row_lengths.count(4)
five_lep = row_lengths.count(5)

print(one_lep, two_lep, three_lep, four_lep, five_lep)

#Lepton count frequency:
#One lepton: 6374629 events
#Two leptons: 647126
#Three leptoons: 6215
#Four leptons: 111
#Five leptons: 3

6374629 647126 6215 111 3


In [ ]:
#Finding the indices of the events where four leptons were observed
def get_indices_lc(my_list, target_value):
    return [i for i, x in enumerate(my_list) if x == target_value]
four_leps = get_indices_lc(row_lengths, 4)
print(four_leps)

NameError: name 'row_lengths' is not defined

In [ ]:
#Code to extract all modules - crashes!! nvm
import numpy as np
import awkward as ak
import uproot
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    'lep_pt', 'lep_n', 'lep_truthMatched','lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]

try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

    print(processed_data_np['lep_pt'][:10])

except Exception as e:
    print(f"Error processing data: {e}")

In [ ]:
#Lepton Charge CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    'lep_charge', #'lep_n', 'lep_truthMatched','lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_charge': list_cat_index0,
    'lep_1_charge': list_cat_index1,
    'lep_2_charge': list_cat_index2,
    'lep_3_charge': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_lep_charge.csv')

df

7028084


,lep_0_charge,lep_1_charge,lep_2_charge,lep_3_charge
0,-1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0
...,...,...,...,...
7028079,-1.0,0.0,0.0,0.0
7028080,-1.0,0.0,0.0,0.0
7028081,1.0,0.0,0.0,0.0
7028082,-1.0,1.0,0.0,0.0


In [ ]:
#Lepton transverse momentum csv maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge',
    'lep_pt', #'lep_eta','lep_phi', 'lep_E', 'lep_type', 'lep_ptcone30', 'lep_etcone20', 'runNumber'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_pt': list_cat_index0,
    'lep_1pt': list_cat_index1,
    'lep_2pt': list_cat_index2,
    'lep_3pt': list_cat_index3,
})

df.to_csv('8TeV_muonmuon_lep_pt.csv')

df

,lep_0_pt,lep_1pt,lep_2pt,lep_3pt
0,40531.855469,0.000000,0.0,0.0
1,37172.488281,0.000000,0.0,0.0
2,41404.363281,0.000000,0.0,0.0
3,36330.359375,0.000000,0.0,0.0
4,29865.917969,0.000000,0.0,0.0
...,...,...,...,...
7028079,86559.039062,0.000000,0.0,0.0
7028080,26242.826172,0.000000,0.0,0.0
7028081,34458.675781,0.000000,0.0,0.0
7028082,73698.351562,27292.826172,0.0,0.0


In [ ]:
#Lepton truthMatched CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n',
    'lep_truthMatched',#'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_truthMatched': list_cat_index0,
    'lep_1_truthMatched': list_cat_index1,
    'lep_2_truthMatched': list_cat_index2,
    'lep_3_truthMatched': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_truthMatched.csv')

df

7028084


,lep_0_truthMatched,lep_1_truthMatched,lep_2_truthMatched,lep_3_truthMatched
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
7028079,0,0,0,0
7028080,0,0,0,0
7028081,0,0,0,0
7028082,0,0,0,0


In [ ]:
#Lepton truthMatched CSV maker
!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched',
    'lep_trigMatched', #'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_trigMatched': list_cat_index0,
    'lep_1_trigMatched': list_cat_index1,
    'lep_2_trigMatched': list_cat_index2,
    'lep_3_trigMatched': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_lep_trigMatched.csv')

df

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.8/393.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.6/919.6 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.7/656.7 kB 41.4 MB/s eta 0:00:00
7028084


,lep_0_trigMatched,lep_1_trigMatched,lep_2_trigMatched,lep_3_trigMatched
0,3,0,0,0
1,3,0,0,0
2,3,0,0,0
3,3,0,0,0
4,1,0,0,0
...,...,...,...,...
7028079,3,0,0,0
7028080,1,0,0,0
7028081,1,0,0,0
7028082,3,1,0,0


In [ ]:
#Lepton type CSV maker
!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched',
    'lep_type', #'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_type': list_cat_index0,
    'lep_1_type': list_cat_index1,
    'lep_2_lep_type': list_cat_index2,
    'lep_3_lep_type': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_lep_type.csv')

df

7028084


,lep_0_type,lep_1_type,lep_2_lep_type,lep_3_lep_type
0,13,0,0,0
1,13,0,0,0
2,13,0,0,0
3,13,0,0,0
4,13,0,0,0
...,...,...,...,...
7028079,13,0,0,0
7028080,13,0,0,0
7028081,13,0,0,0
7028082,13,13,0,0


In [ ]:
#Lepton flag CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge',
    'lep_flag', #'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_flag': list_cat_index0,
    'lep_1_flag': list_cat_index1,
    'lep_2_flag': list_cat_index2,
    'lep_3_flag': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_lep_flag.csv')

df

7028084


,lep_0_flag,lep_1_flag,lep_2_flag,lep_3_flag
0,568344575,0,0,0
1,568344575,0,0,0
2,568344575,0,0,0
3,568344575,0,0,0
4,568344575,0,0,0
...,...,...,...,...
7028079,568342527,0,0,0
7028080,568344575,0,0,0
7028081,568344575,0,0,0
7028082,299909119,299909119,0,0


In [ ]:
#Lepton flag CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag',
    'lep_eta', #'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_eta': list_cat_index0,
    'lep_1_eta': list_cat_index1,
    'lep_2_eta': list_cat_index2,
    'lep_3_eta': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_lep_eta.csv')

df

7028084


,lep_0_eta,lep_1_eta,lep_2_eta,lep_3_eta
0,0.288244,0.000000,0.0,0.0
1,-0.214784,0.000000,0.0,0.0
2,-1.200101,0.000000,0.0,0.0
3,1.624466,0.000000,0.0,0.0
4,1.933381,0.000000,0.0,0.0
...,...,...,...,...
7028079,-1.880404,0.000000,0.0,0.0
7028080,1.954918,0.000000,0.0,0.0
7028081,-0.642925,0.000000,0.0,0.0
7028082,-2.163398,-1.789228,0.0,0.0


In [ ]:
#Lepton phi CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta',
    'lep_phi', #'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_phi': list_cat_index0,
    'lep_1_phi': list_cat_index1,
    'lep_2_phi': list_cat_index2,
    'lep_3_phi': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_lep_phi.csv')

df

7028084


,lep_0_phi,lep_1_phi,lep_2_phi,lep_3_phi
0,1.346999,0.000000,0.0,0.0
1,-2.138006,0.000000,0.0,0.0
2,1.357601,0.000000,0.0,0.0
3,-0.082519,0.000000,0.0,0.0
4,-2.166267,0.000000,0.0,0.0
...,...,...,...,...
7028079,-3.098880,0.000000,0.0,0.0
7028080,-0.419842,0.000000,0.0,0.0
7028081,-2.620870,0.000000,0.0,0.0
7028082,2.341647,-0.212164,0.0,0.0


In [ ]:
#Lepton energy CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi',
    'lep_E', #'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_E': list_cat_index0,
    'lep_1_E': list_cat_index1,
    'lep_2_E': list_cat_index2,
    'lep_3_E': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_lep_E.csv')

df

7028084


,lep_0_E,lep_1_E,lep_2_E,lep_3_E
0,42227.464844,0.000000,0.0,0.0
1,38033.359375,0.000000,0.0,0.0
2,74975.453125,0.000000,0.0,0.0
3,95780.078125,0.000000,0.0,0.0
4,105389.390625,0.000000,0.0,0.0
...,...,...,...,...
7028079,290348.406250,0.000000,0.0,0.0
7028080,94538.742188,0.000000,0.0,0.0
7028081,41829.320312,0.000000,0.0,0.0
7028082,324846.968750,83951.710938,0.0,0.0


In [ ]:
#Lepton z0 CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E',
    'lep_z0', #'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_z0': list_cat_index0,
    'lep_1_z0': list_cat_index1,
    'lep_2_z0': list_cat_index2,
    'lep_3_z0': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_lep_z0.csv')

df

7028084


,lep_0_z0,lep_1_z0,lep_2_z0,lep_3_z0
0,-0.045447,0.000000,0.0,0.0
1,0.225785,0.000000,0.0,0.0
2,-0.030549,0.000000,0.0,0.0
3,0.016618,0.000000,0.0,0.0
4,-0.008122,0.000000,0.0,0.0
...,...,...,...,...
7028079,-0.069141,0.000000,0.0,0.0
7028080,-0.103799,0.000000,0.0,0.0
7028081,-0.014734,0.000000,0.0,0.0
7028082,-0.501517,-0.267624,0.0,0.0


In [ ]:
#Lepton ptcone30 CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0',
    'lep_ptcone30', #'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_ptcone30': list_cat_index0,
    'lep_1_ptcone30': list_cat_index1,
    'lep_2_ptcone30': list_cat_index2,
    'lep_3_ptcone30': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_lep_ptcone30.csv')

df

7028084


,lep_0_ptcone30,lep_1_ptcone30,lep_2_ptcone30,lep_3_ptcone30
0,0.000000,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0
...,...,...,...,...
7028079,5397.591797,0.0,0.0,0.0
7028080,0.000000,0.0,0.0,0.0
7028081,0.000000,0.0,0.0,0.0
7028082,0.000000,0.0,0.0,0.0


In [ ]:
#Lepton etcone20 CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30',
    'lep_etcone20', #'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_etcone20': list_cat_index0,
    'lep_1_etcone20': list_cat_index1,
    'lep_2_etcone20': list_cat_index2,
    'lep_3_etcone20': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_lep_etcone20.csv')

df

7028084


,lep_0_etcone20,lep_1_etcone20,lep_2_etcone20,lep_3_etcone20
0,94.183250,0.000000,0.0,0.0
1,65.886726,0.000000,0.0,0.0
2,628.798279,0.000000,0.0,0.0
3,78.632385,0.000000,0.0,0.0
4,228.175583,0.000000,0.0,0.0
...,...,...,...,...
7028079,773.444763,0.000000,0.0,0.0
7028080,0.000000,0.000000,0.0,0.0
7028081,0.000000,0.000000,0.0,0.0
7028082,-24.844353,673.283325,0.0,0.0


In [ ]:
#Lepton trackd0pvunbiased CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20',
    'lep_trackd0pvunbiased', #'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_trackd0pvunbiased': list_cat_index0,
    'lep_1_trackd0pvunbiased': list_cat_index1,
    'lep_2_trackd0pvunbiased': list_cat_index2,
    'lep_3_trackd0pvunbiased': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_lep_trackd0pvunbiased.csv')

df

7028084


,lep_0_trackd0pvunbiased,lep_1_trackd0pvunbiased,lep_2_trackd0pvunbiased,lep_3_trackd0pvunbiased
0,-0.049129,0.000000,0.0,0.0
1,-0.032381,0.000000,0.0,0.0
2,-0.005932,0.000000,0.0,0.0
3,-0.012364,0.000000,0.0,0.0
4,-0.002284,0.000000,0.0,0.0
...,...,...,...,...
7028079,0.012089,0.000000,0.0,0.0
7028080,-0.015295,0.000000,0.0,0.0
7028081,-0.018467,0.000000,0.0,0.0
7028082,-0.008331,0.023664,0.0,0.0


In [ ]:
#Lepton tracksigd0pvunbiased CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased',
    'lep_tracksigd0pvunbiased', #'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_tracksigd0pvunbiased': list_cat_index0,
    'lep_1_tracksigd0pvunbiased': list_cat_index1,
    'lep_2_tracksigd0pvunbiased': list_cat_index2,
    'lep_3_tracksigd0pvunbiased': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_lep_tracksigd0pvunbiased.csv')

df

7028084


,lep_0_tracksigd0pvunbiased,lep_1_tracksigd0pvunbiased,lep_2_tracksigd0pvunbiased,lep_3_tracksigd0pvunbiased
0,0.015223,0.000000,0.0,0.0
1,0.025348,0.000000,0.0,0.0
2,0.018147,0.000000,0.0,0.0
3,0.017885,0.000000,0.0,0.0
4,0.017784,0.000000,0.0,0.0
...,...,...,...,...
7028079,0.013245,0.000000,0.0,0.0
7028080,0.023698,0.000000,0.0,0.0
7028081,0.016760,0.000000,0.0,0.0
7028082,0.018359,0.020549,0.0,0.0


In [ ]:
#Lepton met_et CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased',
    'met_et', #'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_met_et': list_cat_index0,
    'lep_1_met_et': list_cat_index1,
    'lep_2_met_et': list_cat_index2,
    'lep_3_met_et': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_lep_met_et.csv')

df

7028084


,lep_0_met_et,lep_1_met_et,lep_2_met_et,lep_3_met_et
0,94215.101562,94215.101562,94215.101562,94215.101562
1,30354.056641,30354.056641,30354.056641,30354.056641
2,54632.632812,54632.632812,54632.632812,54632.632812
3,18974.707031,18974.707031,18974.707031,18974.707031
4,18013.089844,18013.089844,18013.089844,18013.089844
...,...,...,...,...
7028079,92065.382812,92065.382812,92065.382812,92065.382812
7028080,10739.869141,10739.869141,10739.869141,10739.869141
7028081,52219.191406,52219.191406,52219.191406,52219.191406
7028082,47151.269531,47151.269531,47151.269531,47151.269531


In [ ]:
#Lepton met_et CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et',
    'met_phi', #'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_met_phi': list_cat_index0,
    'lep_1_met_phi': list_cat_index1,
    'lep_2_met_phi': list_cat_index2,
    'lep_3_met_phi': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_lep_met_phi.csv')

df

7028084


,lep_0_met_phi,lep_1_met_phi,lep_2_met_phi,lep_3_met_phi
0,-1.394356,-1.394356,-1.394356,-1.394356
1,0.325955,0.325955,0.325955,0.325955
2,-2.053428,-2.053428,-2.053428,-2.053428
3,2.315790,2.315790,2.315790,2.315790
4,0.869603,0.869603,0.869603,0.869603
...,...,...,...,...
7028079,-2.127335,-2.127335,-2.127335,-2.127335
7028080,3.076264,3.076264,3.076264,3.076264
7028081,0.605798,0.605798,0.605798,0.605798
7028082,0.077038,0.077038,0.077038,0.077038


In [ ]:
#Lepton jet_n CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi',
    'jet_n', #'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_met_jet_n': list_cat_index0,
    'lep_1_met_jet_n': list_cat_index1,
    'lep_2_met_jet_n': list_cat_index2,
    'lep_3_met_jet_n': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_lep_jet_n.csv')

df

7028084


,lep_0_met_jet_n,lep_1_met_jet_n,lep_2_met_jet_n,lep_3_met_jet_n
0,0,0,0,0
1,1,1,1,1
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
7028079,1,1,1,1
7028080,0,0,0,0
7028081,0,0,0,0
7028082,3,3,3,3


In [ ]:
#Lepton alljet_n CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n',
    'alljet_n', #'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_met_all_jet_n': list_cat_index0,
    'lep_1_met_alljet_n': list_cat_index1,
    'lep_2_met_alljet_n': list_cat_index2,
    'lep_3_met_alljet_n': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_lep_all_jet_n.csv')

df

7028084


,lep_0_met_all_jet_n,lep_1_met_alljet_n,lep_2_met_alljet_n,lep_3_met_alljet_n
0,0,0,0,0
1,1,1,1,1
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
7028079,1,1,1,1
7028080,0,0,0,0
7028081,0,0,0,0
7028082,3,3,3,3


In [ ]:
#Lepton jet_pt CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n',
    'jet_pt', #'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_met_jet_pt': list_cat_index0,
    'lep_1_met_jet_pt': list_cat_index1,
    'lep_2_met_jet_pt': list_cat_index2,
    'lep_3_met_jet_pt': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_jet_pt.csv')

df

7028084


,lep_0_met_jet_pt,lep_1_met_jet_pt,lep_2_met_jet_pt,lep_3_met_jet_pt
0,0.000000,0.000000,0.000000,0.0
1,27929.968750,0.000000,0.000000,0.0
2,0.000000,0.000000,0.000000,0.0
3,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.0
...,...,...,...,...
7028079,130350.367188,0.000000,0.000000,0.0
7028080,0.000000,0.000000,0.000000,0.0
7028081,0.000000,0.000000,0.000000,0.0
7028082,70982.773438,55891.566406,25352.851562,0.0


In [ ]:
#Lepton jet_eta CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt',
    'jet_eta', #'jet_phi', 'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_met_jet_eta': list_cat_index0,
    'lep_1_met_jet_eta': list_cat_index1,
    'lep_2_met_jet_eta': list_cat_index2,
    'lep_3_met_jet_eta': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_jet_eta.csv')

df

7028084


,lep_0_met_jet_eta,lep_1_met_jet_eta,lep_2_met_jet_eta,lep_3_met_jet_pt
0,0.000000,0.000000,0.000000,0.0
1,-2.170531,0.000000,0.000000,0.0
2,0.000000,0.000000,0.000000,0.0
3,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.0
...,...,...,...,...
7028079,-1.303018,0.000000,0.000000,0.0
7028080,0.000000,0.000000,0.000000,0.0
7028081,0.000000,0.000000,0.000000,0.0
7028082,-1.036580,-0.559905,-1.185488,0.0


In [ ]:
#Lepton jet_phi CSV maker
!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta',
    'jet_phi', #'jet_E', 'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_met_jet_phi': list_cat_index0,
    'lep_1_met_jet_phi': list_cat_index1,
    'lep_2_met_jet_phi': list_cat_index2,
    'lep_3_met_jet_phi': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_jet_phi.csv')

df

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.8/393.8 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.6/919.6 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.7/656.7 kB 28.8 MB/s eta 0:00:00
7028084


,lep_0_met_jet_phi,lep_1_met_jet_phi,lep_2_met_jet_phi,lep_3_met_jet_phi
0,0.000000,0.000000,0.000000,0.0
1,1.423280,0.000000,0.000000,0.0
2,0.000000,0.000000,0.000000,0.0
3,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.0
...,...,...,...,...
7028079,0.133568,0.000000,0.000000,0.0
7028080,0.000000,0.000000,0.000000,0.0
7028081,0.000000,0.000000,0.000000,0.0
7028082,-1.719984,0.949665,-2.344595,0.0


In [ ]:
#Lepton jet_E CSV maker
!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi',
    'jet_E', #'jet_m', 'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_jet_E': list_cat_index0,
    'lep_1_jet_E': list_cat_index1,
    'lep_2_jet_E': list_cat_index2,
    'lep_3_jet_E': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_jet_E.csv')

df

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.8/393.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.6/919.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.7/656.7 kB 30.3 MB/s eta 0:00:00
7028084


,lep_0_jet_E,lep_1_jet_E,lep_2_jet_E,lep_3_jet_E
0,0.000000,0.000000,0.000000,0.0
1,124052.250000,0.000000,0.000000,0.0
2,0.000000,0.000000,0.000000,0.0
3,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.0
...,...,...,...,...
7028079,257650.250000,0.000000,0.000000,0.0
7028080,0.000000,0.000000,0.000000,0.0
7028081,0.000000,0.000000,0.000000,0.0
7028082,113259.226562,65126.238281,45680.523438,0.0


In [ ]:
#Lepton jet_m CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E',
    'jet_m', #'jet_jvf', 'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_jet_m': list_cat_index0,
    'lep_1_jet_m': list_cat_index1,
    'lep_2_jet_m': list_cat_index2,
    'lep_3_jet_m': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_jet_m.csv')

df

7028084


,lep_0_jet_m,lep_1_jet_m,lep_2_jet_m,lep_3_jet_m
0,0.000000,0.000000,0.00000,0.0
1,4573.632812,0.000000,0.00000,0.0
2,0.000000,0.000000,0.00000,0.0
3,0.000000,0.000000,0.00000,0.0
4,0.000000,0.000000,0.00000,0.0
...,...,...,...,...
7028079,6072.829590,0.000000,0.00000,0.0
7028080,0.000000,0.000000,0.00000,0.0
7028081,0.000000,0.000000,0.00000,0.0
7028082,11658.162109,5615.926758,5445.79834,0.0


In [ ]:
#Lepton jet_jvf CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m',
    'jet_jvf', #'jet_trueflav', 'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_jet_jvf': list_cat_index0,
    'lep_1_jet_jvf': list_cat_index1,
    'lep_2_jet_jvf': list_cat_index2,
    'lep_3_jet_jvf': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_jet_jvf.csv')

df

7028084


,lep_0_jet_jvf,lep_1_jet_jvf,lep_2_jet_jvf,lep_3_jet_jvf
0,0.000000,0.000000,0.0,0.0
1,0.952346,0.000000,0.0,0.0
2,0.000000,0.000000,0.0,0.0
3,0.000000,0.000000,0.0,0.0
4,0.000000,0.000000,0.0,0.0
...,...,...,...,...
7028079,0.902416,0.000000,0.0,0.0
7028080,0.000000,0.000000,0.0,0.0
7028081,0.000000,0.000000,0.0,0.0
7028082,0.963319,0.876117,1.0,0.0


In [ ]:
#Lepton jet_trueflav CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf',
    'jet_trueflav', #'jet_truthMatched', 'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_jet_trueflav': list_cat_index0,
    'lep_1_jet_trueflav': list_cat_index1,
    'lep_2_jet_trueflav': list_cat_index2,
    'lep_3_jet_trueflav': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_jet_trueflav.csv')

df

7028084


,lep_0_jet_trueflav,lep_1_jet_trueflav,lep_2_jet_trueflav,lep_3_jet_trueflav
0,0,0,0,0
1,-99,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
7028079,-99,0,0,0
7028080,0,0,0,0
7028081,0,0,0,0
7028082,-99,-99,-99,0


In [ ]:
#Lepton jet_truthMatched CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf','jet_trueflav',
    'jet_truthMatched', #'jet_SV0', 'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_jet_truthMatched': list_cat_index0,
    'lep_1_jet_truthMatched': list_cat_index1,
    'lep_2_jet_truthMatched': list_cat_index2,
    'lep_3_jet_truthMatched': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_jet_truthMatched.csv')

df

7028084


,lep_0_jet_truthMatched,lep_1_jet_truthMatched,lep_2_jet_truthMatched,lep_3_jet_truthMatched
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
7028079,0,0,0,0
7028080,0,0,0,0
7028081,0,0,0,0
7028082,0,0,0,0


In [ ]:
#Lepton jet_SV0 CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf','jet_trueflav', 'jet_truthMatched',
    'jet_SV0', #'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_jet_SV0': list_cat_index0,
    'lep_1_jet_SV0': list_cat_index1,
    'lep_2_jet_SV0': list_cat_index2,
    'lep_3_jet_SV0': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_jet_SV0.csv')

df

7028084


,lep_0_jet_SV0,lep_1_jet_SV0,lep_2_jet_SV0,lep_3_jet_SV0
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
...,...,...,...,...
7028079,0.0,0.0,0.0,0.0
7028080,0.0,0.0,0.0,0.0
7028081,0.0,0.0,0.0,0.0
7028082,0.0,0.0,0.0,0.0


In [ ]:
#Lepton jet_MV1 CSV maker
#!pip install uproot

import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import pandas as pd


file_path = '/content/local_data/Data/DataMuons.root'

branches_to_extract = [
    #'lep_charge', 'lep_n', 'lep_truthMatched', 'lep_trigMatched', 'lep_type', 'lep_charge', 'lep_flag', 'lep_eta', 'lep_phi', 'lep_E', 'lep_z0', 'lep_ptcone30', 'lep_etcone20', 'lep_trackd0pvunbiased', 'lep_tracksigd0pvunbiased', 'met_et', 'met_phi', 'jet_n', 'alljet_n', 'jet_pt', 'jet_eta', 'jet_phi', 'jet_E', 'jet_m', 'jet_jvf','jet_trueflav', 'jet_truthMatched', 'jet_SV0',
    'jet_MV1'
]
try:
    with uproot.open(file_path) as file:
        tree = file['mini;1']
        extracted_data = tree.arrays(branches_to_extract, library='ak')

    num_events = len(extracted_data)
    fixed_lepton_count = 4
    processed_data_np = {}
    print(num_events)

    for branch_name in branches_to_extract:
        ak_array = extracted_data[branch_name]
        is_jagged_array = ak_array.ndim > 1

        if is_jagged_array:
            padded_ak_array = ak.fill_none(ak.pad_none(ak_array, fixed_lepton_count, clip=True), 0)
            processed_data_np[branch_name] = ak.to_numpy(padded_ak_array)
        else:
            repeated_scalar = np.repeat(ak.to_numpy(ak_array), fixed_lepton_count).reshape(num_events, fixed_lepton_count)
            processed_data_np[branch_name] = repeated_scalar

except Exception as e:
    print(f"Error processing data: {e}")

#Making the awkward arrary into a list
python_list = ak.to_list(processed_data_np)

#Creating lepton indexes
list_cat_index0 = []
list_cat_index1 = []
list_cat_index2 = []
list_cat_index3 = []

# Assign a lepton index for each lepton in the event
for array_data in python_list.values():
  for row in array_data:
    list_cat_index0.append(row[0])
    list_cat_index1.append(row[1])
    list_cat_index2.append(row[2])
    list_cat_index3.append(row[3])

# Make array into into a pd.DataFrame and specify columns
df = pd.DataFrame({
    'lep_0_jet_MV1': list_cat_index0,
    'lep_1_jet_MV1': list_cat_index1,
    'lep_2_jet_MV1': list_cat_index2,
    'lep_3_jet_MV1': list_cat_index3,
})

df.to_csv('8TeV_Muonmuon_jet_MV1.csv')

df

7028084


,lep_0_jet_MV1,lep_1_jet_MV1,lep_2_jet_MV1,lep_3_jet_MV1
0,0.000000,0.000000,0.000000,0.0
1,0.055552,0.000000,0.000000,0.0
2,0.000000,0.000000,0.000000,0.0
3,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.0
...,...,...,...,...
7028079,0.056128,0.000000,0.000000,0.0
7028080,0.000000,0.000000,0.000000,0.0
7028081,0.000000,0.000000,0.000000,0.0
7028082,0.054500,0.057929,0.256593,0.0
